# Paper Crane Data Analysis

Notebook Structure:
1. Code needed to perform the analysis (can be skipped)
2. Findings Presentation
3. Addendum with full outputs

Notebook notation:
- '# main header for the notebook:
- '## #.                     Topic header
- '### #.#                   Section header
- '#### #.#.#                Subsection Header

## 1. Data preparation

The section connects and fetches the data for the next stages 

### 1.1 Importing Libraries

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns

In [ ]:
# Setting so that all rows are dispalyed for analysis
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### 1.2 Importing Data

In [ ]:
# Outlining the connection string
Driver = '{SQL Server}'
server = '' 
database = ''

In [ ]:
# Credentials
uid = ''
psswd = ''

In [ ]:
time.sleep(5)

In [ ]:
# making the connection to the server
while True:
    try:
        conn = pyodbc.connect(f'Driver={Driver};Server={server};Database={database};uid={uid};pwd={psswd}')
    except pyodbc.OperationalError:
        continue
    break

In [ ]:
# Loading Datasets
sql = 'SELECT * FROM database.table'
analysed_data = pd.read_sql(sql, conn)
sql1 = 'SELECT * FROM database.table'
attom_data = pd.read_sql(sql1, conn)

### 1.3 Running analysis

This section is used to run the analysis and can be ignored by most consumers

#### 1.3.1 Match on ID 

In [ ]:
# match up the Id's
attom_id = pd.DataFrame(attom_data['ATTOMID'])
attom_id.columns = ['attomid']
pc_id = pd.DataFrame(analysed_data['attomid'].unique())
pc_id.columns = ['attomid']
merge_id = pd.merge(attom_id, pc_id, how='inner', on='attomid')

In [ ]:
# get distinct attomid with state from paper crane data
distinct_states = analysed_data[['attomid', 'situsstate']].drop_duplicates()

# get counts by value
state_at = attom_data['SitusState'].value_counts()
state_pc = distinct_states['situsstate'].value_counts()

# transform to dataframe
state_at = pd.DataFrame([(key, item) for key, item in state_at.items()])
state_at.columns = ['state', 'attomCount']
state_pc = pd.DataFrame([(key, item) for key, item in state_pc.items()])
state_pc.columns = ['state', 'paperCraneCount']

# combine the above datafreames into one
state_merge = pd.merge(state_at, state_pc, how='outer', on='state')
state_merge['matchRate'] = state_merge['paperCraneCount'] / state_merge['attomCount']
sorted_StateMatch = state_merge.sort_values(by=['matchRate'], ascending=False)

In [ ]:
# Variables to show match on ID visual
matched = len(merge_id) / len(attom_id)
matchLabels = ['Matched', 'Unmatched']
datapoints = [matched, 1-matched]

#### 1.3.2 Categorical and Continous Columns 

In [ ]:
# Update dataset to change the date format column type from object to datetime
mask = analysed_data.astype(str).apply(lambda x : x.str.match(r'(\d{2,4}-\d{2}-\d{2,4})+').all())
analysed_data.loc[:,mask] = analysed_data.loc[:,mask].apply(pd.to_datetime)

In [ ]:
# Categorical, Continuous and Date column lists
cat_columns = [column for column in analysed_data.columns if analysed_data[column].dtype == 'object']
cont_columns = [column for column in analysed_data.columns if analysed_data[column].dtype != 'object']
date_columns = [column for column in analysed_data.columns if str(analysed_data[column].dtype)[:4] == 'date']

# move the bool columns from the cont to cat list
bool_columns = [column for column in analysed_data.columns if analysed_data[column].dtype == 'bool']
cat_columns = cat_columns + bool_columns
cont_columns = list(set(cont_columns) - set(bool_columns))

In [ ]:
date_formats = ['nn/nn/nnnn', 'nnnn/nn/nn', 'nn/nn/nn', 'n/n/nn']

In [ ]:
# Loop through the Object columns and checking for date formats in the list 
# Only take the first 100 examples of the column for format
# Creating an array and looping through object columns
matched_formats = {}
for column in cat_columns:
    test_array = np.array(analysed_data[column][:100])
    format_list = []
    for cell in test_array:
        if cell != None:
            cell_format = ''
            for character in str(cell):
                if character.isnumeric() == True:
                    cell_format += 'n'
                elif character.isalpha() == True:
                    cell_format += 'l'
                else:
                    cell_format += '/'
        if cell_format not in format_list:
            format_list.append(cell_format)
    test_array = []
    if len(format_list) == 1:
        for dformat in date_formats:
            if dformat == format_list[0][:len(dformat)] and column not in matched_formats:                    
                matched_formats[column] = dformat
            elif dformat == format_list[0][:len(dformat)] and column in matched_formats:
                if len(dformat) > len(matched_formats[column]):
                    matched_formats[column] = dformat
            else:
                pass
date_columns = date_columns + list(matched_formats.keys())            

In [ ]:
# Remove date_columns list from cat_columns
cat_columns = list(set(cat_columns) - set(date_columns))

In [ ]:
# Column types for the dataset
column_list = analysed_data.dtypes
dataset_columns = pd.DataFrame([(key, item) for key, item in column_list.items()])
dataset_columns.columns = ['ColumnName', 'ColumnType']

# Generate the count of instance by column type
colType_count = dataset_columns['ColumnType'].value_counts()
colType = np.array([(str(key)) for key, item in colType_count.items()])
colCount = np.array([(item) for key, item in colType_count.items()])

#### 1.3.3. Basic Stats for Categorical and Continous Columns 

In [ ]:
# Stats for categorical columns
cat_stats = analysed_data[cat_columns].describe().transpose()

In [ ]:
# Dataset stats for continuous columns
cont_stats = analysed_data[cont_columns].describe().transpose().apply(lambda s: s.apply('{0:.5f}'.format))

#### 1.3.4 NULL, 0, Distinct count by Columns 

In [ ]:
# count number of istances of a value
null_count = analysed_data.isna().sum()
zero_count = (analysed_data == 0).astype(int).sum(axis=0)

# convert into a dataframe
null_count_df = pd.DataFrame([(key, item) for key, item in null_count.items()])
null_count_df.columns = ['ColumnName', 'nullCount']

zero_count_df = pd.DataFrame([(key, item) for key, item in zero_count.items()])
zero_count_df.columns = ['ColumnName', 'zeroCount']

full_count = pd.merge(null_count_df, zero_count_df, how='outer', on='ColumnName')

# counting up unique values in each column
unique_val = {}
for col in analysed_data:
    unique_val[col] = analysed_data[col].unique()

distinct_values = []
for key, item in unique_val.items():
    distinct_values.append([key, len(item)])
distinct_values = pd.DataFrame(distinct_values)
distinct_values.columns = ['ColumnName','DistinctCount']

full_count = pd.merge(full_count, distinct_values, how='outer', on='ColumnName').fillna(0)

In [ ]:
# How many of zeros and NULLS are there in the entire dataset?
nulls = full_count['nullCount'].sum()
zeros = full_count['zeroCount'].sum()

# How many cells are there in the dataset in total
all_cells = analysed_data.shape[0] * analysed_data.shape[1]

# The values for pie chart that will be displayed in secion 2
labels = 'Filled Cells', 'Zeros', 'NULLs'
sizes = [all_cells - zeros - nulls, zeros, nulls]
explode = [0.2, 0, 0]

#### 1.3.5 Correlation Calculations 

In [ ]:
# calculating the correlation of the dataset
corr_data = analysed_data.corr()

In [ ]:
# Unstacked Correlation analysis
corr_data_us = corr_data.unstack()
sorted_corr = corr_data_us.sort_values(ascending=False)

# move the unstacked data into a list
corr_list = [(key[0], key[1], item) for key, item in sorted_corr.items()]

# remove the correlation with thyself
inx = 0
test_list = []
while inx < len(corr_list):
    if corr_list[inx][0] == corr_list[inx][1]:
        test_list.append(corr_list[inx])
        corr_list.remove(corr_list[inx])
        inx -= 1
    inx += 1
    
corr_list_df = pd.DataFrame(corr_list)
corr_list_df.columns = ['ColumnName1', 'ColumnName2', 'Correlation']
corr_list_df = corr_list_df.dropna()

####  1.3.6 String Length

In [ ]:
# block creates a DataFrame with the value lenght
value_length = {}
for name in dataset_columns[dataset_columns['ColumnType'] != 'bool']['ColumnName']:
    value_length[name] = [len(str(var)) for var in analysed_data[name]]
value_lengthDF = pd.DataFrame(value_length)

del value_length

In [ ]:
# block creates DataFrame with average, min and max lenght of string for each column
columnLenght_info = {}
for name in value_lengthDF.columns:
    columnLenght_info[name] = [np.average(value_lengthDF[name]),np.amin(value_lengthDF[name]),
                               np.amax(value_lengthDF[name])]
columnLenghtDF = pd.DataFrame(columnLenght_info).transpose()
columnLenghtDF.columns = ['average', 'minimum', 'maximum']

del columnLenght_info

#### 1.3.7 Dataset characteristics 

In [ ]:
# block creates DataFrame for describe function for continuous and categorical columns
contDesc = analysed_data[cont_columns].describe().apply(lambda s: s.apply('{0:.1f}'.format)).transpose()
catDesc = analysed_data[cat_columns].describe().transpose()

In [ ]:
# Columns with some negative values
someNegColumns = contDesc.loc[contDesc['min'] < '0'][['mean', 'min', 'max']]

# Columns with all negative values
allNegColumns = contDesc.loc[contDesc['max'] < '0'][['mean', 'min', 'max']]

# Combining the two DataFrames
colWithNegValues = pd.concat([someNegColumns, allNegColumns], axis=1, sort=True)

In [ ]:
# Block creates a DataFrame with the column Name and % of negative values in it
negValCount = {}
for name in cont_columns:
    instance = 0
    for val in analysed_data[name]:
        if val < 0:
            instance += 1
    if instance > 0:
        negValCount[name] = instance / analysed_data.shape[0] * 100

# creates lists for 2 columns 
negValKey, negValItem = [], []
for key, item in negValCount.items():
    negValKey.append(key)
    negValItem.append(item)

# building the DataFrame to for Negative Value % of total
negValCountDF = pd.DataFrame({'ColumnName': negValKey, 'Neg Val %': negValItem})
negValCountDF = negValCountDF.sort_values(by=['Neg Val %'], ascending=False)
    
# Removing the lists
del negValKey
del negValItem    

#### 1.3.8 Date Columns 

In [ ]:
# DataFrame for the date columns
if len(date_columns) > 0:
    dateDescribe = analysed_data[date_columns].describe().transpose()
else:
    dateDescribe = []

## 2. Findings Presentation 

Section contains minimal amount of code and is used to present the results

###  2.1 Basic dataset information

#### 2.1.1 Dataset Shape

In [ ]:
# Dataset shape
print(f'The dataset has {analysed_data.shape[0]:,} rows and {analysed_data.shape[1]:,} and columns')

#### 2.1.2 Columns by Type

In [ ]:
print(f'There are {len(cat_columns)} categorical columns and {len(cont_columns)} numerical columns in the dataset')

### 2.2 Data Set Match

In [ ]:
print(f'Match Rate is {matched:.2%}')
plt.pie(x=datapoints, labels=matchLabels, autopct='%1.2f%%')
plt.show()

### 2.3 Dataset Match by State

In [ ]:
# output the match rates by state
plt.bar(sorted_StateMatch['state'], sorted_StateMatch['matchRate'])
plt.show()

### 2.4 Columns by Type 

In [ ]:
plt.bar(colType, colCount)
plt.xlabel('Column Category')
plt.ylabel('Instance count')
plt.show()

In [ ]:
print(f'{colCount[0] / analysed_data.shape[1]:.2%} of all columns are numerical')

###  2.5 NULL and 0 values

In [ ]:
print(f'There are {nulls:,} cells with NULL value and {zeros:,} cells with 0 value')
print(f'A {nulls/all_cells:,.2%} of cells have NULL value and {zeros/all_cells:,.2%} of cells have 0 value')

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, explode=explode, autopct='%1.1f%%')
plt.show

### 2.6 Column Metrics

#### 2.6.1 Negative Values 

In [ ]:
negValCountDF

In [ ]:
g = sns.barplot(x=negValCountDF['Neg Val %'], y=negValCountDF['ColumnName'], data=negValCountDF)

#### 2.6.2 Date Columns

Date columns in the dataset with column type

In [ ]:
if len(date_columns) > 0:
    analysed_data[date_columns].describe() 
else:
    print('\n No date columns were detected in the dataset')

Date Ranges in each column

In [ ]:
if len(date_columns) > 0:
    dateDescribe
else:
    print('')

## 3. Addendum

### 3.1 Top 10 Rows of Dataset

In [ ]:
analysed_data.head()

### 3.2 Dataset Details

#### 3.2.1 Column Types

In [ ]:
dataset_columns

####  3.2.2 Column Lenghts

The column lenghts are sorted largest to smallest

In [ ]:
columnLenghtDF.sort_values(by=['maximum'], ascending=False)

### 3.3. Columns with NULL, Zero and distinct counts

In [ ]:
full_count

### 3.4 Correlation Matrix

#### 3.4.1 Correlation Matrix for Continous Columns

In [ ]:
plt.subplots(figsize=(10,10))
ax = sns.heatmap(corr_data, vmin=-1, vmax=1, center=0,
                 cmap=sns.diverging_palette(20,220,n=200),
                 square=True
                )
ax.set_xticklabels(ax.get_xticklabels(),
                  rotation=45,
                  horizontalalignment='right')
plt.show()

In [ ]:
corr_data

#### 3.4.2 Highest correlated fields (positive)

In [ ]:
correlation_threshold = 0.9

In [ ]:
highest_positive_corr = corr_list_df.loc[corr_list_df['Correlation'] > correlation_threshold]
highest_positive_corr.sort_values(by = ['Correlation'], ascending=False)

####  3.4.3 Highest correlated fields (negative)

In [ ]:
highest_negative_corr = corr_list_df.loc[corr_list_df['Correlation'] < -correlation_threshold]
highest_negative_corr.sort_values(by = ['Correlation'], ascending=True)

###  3.5 Histograms

#### 3.5.1 Histograms for Continous Columns

Histograms for all continous columns. The graphs are aranged in alphabetical order. Below the graph is a list of columns

In [ ]:
col_grid = 5
row_grid = (len(cont_columns) // col_grid) + 1
fig = analysed_data[cont_columns].hist(bins=50, figsize=(15,40), layout=(row_grid, col_grid))
plt.show()

In [ ]:
for idx, col in enumerate(cont_columns):
    print(f'{idx + 1} {col}')